In [1]:
import os

os.chdir(r'../../../учёба/09s/reasoning/onto-work/c_owl')

In [2]:
os.getcwd()

'C:\\D\\Нинь\\учёба\\09s\\reasoning\\onto-work\\c_owl'

In [3]:
from pycparser import c_parser, c_ast, parse_file

In [4]:
import c2onto

definitions OK


In [5]:
from importlib import reload

In [21]:
# Перезагрузка файла
c2onto = reload(c2onto)

definitions OK


In [7]:
uniq_names = set()
suffix_sep = '_'

def ensure_unique(name):
    if name in uniq_names:
        i = 1
        new_name = name+suffix_sep+str(i)
        while new_name in uniq_names:
            i += 1
            new_name = name+suffix_sep+str(i)
        name = new_name
    
    uniq_names.add(name)
    return name

In [8]:
print(ensure_unique('no-name'))
print(ensure_unique('no-name'))
print(ensure_unique('no-name'))

no-name
no-name_1
no-name_2


In [9]:
OWLPredicate = {
    "type" : 'rdf:type',
}

In [22]:
if __name__ == "__main__":
    
    c2onto.uniq_names.clear()
    
#     alg_filename = 'examples/ex2f.c'
#     alg_filename = 'examples/ex-seq.c'
    alg_filename = 'examples/ex-test.c'
    
    ast = parse_file(alg_filename, use_cpp=False)
    
    alg = c2onto.Algorithm(ast)
#     function_defs = find_func_defs(ast)
    
#     if not function_defs:
#     	print('no functions detected.')
#     	exit(0)
    
#     for fd in function_defs:
#     	print()
#     	fd.show()


go()
f()
main()


In [11]:
triples = alg.get_triples()
print()
print(len(triples))
str(triples)


16


"[('Algorithm', 'rdf:type', 'Algorithm'), ('Algorithm', 'hasFunc', 'main()'), ('main()', 'rdf:type', 'Function'), ('main()', 'hasName', 'main'), ('main()', 'hasBody', 'Seq_1'), ('Seq_1', 'rdf:type', 'Block'), ('Seq_1', 'hasFirstSt', 'call-hello()'), ('Seq_1', 'hasLastSt', 'call-bye()'), ('Seq_1', 'hasSubStmt', 'call-hello()'), ('call-hello()', 'rdf:type', 'FuncCall'), ('Seq_1', 'hasSubStmt', 'call-how_are_you()'), ('call-how_are_you()', 'rdf:type', 'FuncCall'), ('call-hello()', 'hasNextSt', 'call-how_are_you()'), ('Seq_1', 'hasSubStmt', 'call-bye()'), ('call-bye()', 'rdf:type', 'FuncCall'), ('call-how_are_you()', 'hasNextSt', 'call-bye()')]"

In [15]:
print('\n'.join(map(str, triples)))

('Algorithm', 'rdf:type', 'Algorithm')
('Algorithm', 'hasFunc', 'main()')
('main()', 'rdf:type', 'Function')
('main()', 'hasName', 'main')
('main()', 'hasBody', 'Seq_1')
('Seq_1', 'rdf:type', 'Block')
('Seq_1', 'hasFirstSt', 'call-hello()')
('Seq_1', 'hasLastSt', 'call-bye()')
('Seq_1', 'hasSubStmt', 'call-hello()')
('call-hello()', 'rdf:type', 'FuncCall')
('Seq_1', 'hasSubStmt', 'call-how_are_you()')
('call-how_are_you()', 'rdf:type', 'FuncCall')
('call-hello()', 'hasNextSt', 'call-how_are_you()')
('Seq_1', 'hasSubStmt', 'call-bye()')
('call-bye()', 'rdf:type', 'FuncCall')
('call-how_are_you()', 'hasNextSt', 'call-bye()')


## Определения функций и классов для трассы

In [100]:
def parse_trace_token(token):
    """ parse_trace_token("name#N") -> ("name", N) 
    in absence of #N ssuffix, N is set to 0. """
    chunks = token.split('#')
#     print(chunks)
    if len(chunks) == 2:
        name, N = chunks
        try: N = int(N)
        except Exception: raise SyntaxError("Wrong number suffix: '%s' in token '%s'." % (N, token))
        return (name, N)
    if len(chunks) == 1:
        N = 0
        print("Warning: number suffix is missing, using %d as default for token '%s'." % (N, token))
        return (chunks[0], N)  # a chunk, and 0 as N
    else:
        raise SyntaxError("Unrecognized 'name#N' token: '%s'." % token)
    

In [96]:
class TraceNode:
    """Base for an trace node. """
    def __init__(self, type_name='stmt', parent=None, alg_node=None, N=-1, attributes=None):
        self.type_name = type_name
        self.parent = parent
        self.alg_node = alg_node
        self.N = N
        self.attributes = attributes or dict()
        # calculated attributes
        self.make_node_name(name=attributes["name"] if (attributes and "name" in attributes) else None)
        print('=== %20s created.' % self.node_name)
    def make_node_name(self, name=None, N=None):
        self.node_name = "%s" % ensure_unique(name or self.type_name)
        if N is None and self.N > -1:
            N = self.N
#         N = str(N)
        if N is not None:
            self.node_name += "#" + str(N)
    def search_up(self, node_class):
        if isinstance(self, node_class): return self
        elif self.parent:   return self.parent.search_up(node_class)
        else: print('!!! %s not found in tree!' % node_class.__name__); return None
    def get_triples(self):
        triples = []
        if self.alg_node:
            triples += [
                    (self.node_name, "hasOrigin", self.alg_node.node_name),
            ]
        if self.N > -1:
            triples += [
                    (self.node_name, "hasN", self.N),
            ]
        return triples
    

class Trace(TraceNode):
    """ Root for trace tree created from .tr file """
    def __init__(self, alg_root=None):
        TraceNode.__init__(self, type_name='Trace',
                              alg_node=alg_root,
                              parent = None,
                             )
        self.acts = []
        self.contexts = []
        # {name_str -> obj}
        self.act_names = dict()
        self.context_names = dict()
    def parse_act_token(self, token):
        if token in self.act_names:
            return self.act_names[token]
        else:
            name, N = parse_trace_token(token)
            # заглушка!
            act = ActNode(self, name=name, N=N)
            self.act_names[token] = act
            self.acts.append(act)
            return act
        
    def parse_context_label(self, token):
        if token in self.context_names:
            return self.context_names[token]
        else:
            name, N = parse_trace_token(token)
            # заглушка!
            ctx = ActContextNode(self, name=name, N=N)
            self.context_names[token] = ctx
            self.contexts.append(ctx)
            return ctx
        
    def get_triples(self):
        triples = [
            (self.node_name, OWLPredicate["type"], self.type_name),
        ]
        # call parent
        triples += TraceNode.get_triples(self)
        if self.acts:
            triples += [
                (self.node_name, "hasFirstAct", self.acts[0].node_name),
                (self.node_name, "hasLastAct", self.acts[-1].node_name),
            ]
        prev_act = None
        for act in self.acts:
            triples += [
                (self.node_name, "hasAct", act.node_name),
            ]
            triples += act.get_triples()
            if prev_act:
                triples += [
                    (prev_act.node_name, "hasNextAct", act.node_name),
                ]
            prev_act = act
        return triples
    
class ActNode(TraceNode):
    """ Act of trace """
    def __init__(self, parent, alg_node=None, name='unkn_act', N=-1, begin_labels=None):
        TraceNode.__init__(self, type_name='Act',
                              alg_node=alg_node,
                              parent = parent,
                              N = N,
                              attributes={ "name":name, }
                          )
        self.begin_labels = begin_labels or []
        self.end_labels = []
    def get_triples(self):
        triples = [
            (self.node_name, OWLPredicate["type"], self.type_name),
        ]
        # call parent
        triples += TraceNode.get_triples(self)
        for lbl in self.begin_labels:
            triples += [
                (self.node_name, "hasBeginLabel", lbl.node_name),
            ]
            triples += lbl.get_triples()
        for lbl in self.end_labels:
            triples += [
                (self.node_name, "hasEndLabel", lbl.node_name),
            ]
            triples += lbl.get_triples()
        return triples
    
class ActContextNode(TraceNode):
    """ Context (compound Act) can be labelled with BeginLabel and/or EndLabel """
    def __init__(self, parent, alg_node=None, name='unkn_context', N=-1):
        TraceNode.__init__(self, type_name='ActContext',
                              alg_node=alg_node,
                              parent = parent,
                              N = N,
                              attributes={ "name":name, }
                          )
        self.begin = None
        self.end = None
    def get_begin(self):
        if not self.begin:
            self.begin = BeginLabelNode(self)
        return self.begin
    def get_end(self):
        if not self.end:
            self.end = EndLabelNode(self)
        return self.end
    def get_triples(self):
        triples = [
            (self.node_name, OWLPredicate["type"], self.type_name),
        ]
        # call parent
        triples += TraceNode.get_triples(self)
        return triples
    
class AbstractLabelNode(TraceNode):
    """ Abstract Act's label """
    def __init__(self, parent, type_name='Any~Label'):
        assert isinstance(parent, ActContextNode), 'Label must be connected to ActContextNode via parent parameter.'
        TraceNode.__init__(self, type_name=type_name,
                              parent = parent,
                              attributes={ "name":parent.node_name+'_'+type_name, }
                          )
    def get_triples(self):
        triples = [
            (self.node_name, OWLPredicate["type"], self.type_name),
            (self.node_name, "isLabelOf", self.parent.node_name),
        ]
        # call parent  -  unnesessary (no appropriate properties assigned)
#         triples += TraceNode.get_triples(self)
        # call context
        triples += self.parent.get_triples()
        #         # call context, & prevent duplicates
        #         parent_triples = self.parent.get_triples()
        #         for pt in parent_triples:
        #             if pt not in triples:
        #                 triples.append(pt)
        return triples
    
class BeginLabelNode(AbstractLabelNode):
    """ Act's labell: begin of Context """
    def __init__(self, parent):
        AbstractLabelNode.__init__(self, type_name='BeginLabel',  parent = parent)
class EndLabelNode(AbstractLabelNode):
    """ Act's label: end of Context """
    def __init__(self, parent):
        AbstractLabelNode.__init__(self, type_name='EndLabel', parent = parent)


## Прочитаем трассу из файла

In [97]:
trace_filename = 'examples/ex-seq.tr'

with open(trace_filename) as f:
    lines = f.readlines()
tokens = [L.strip().split() for L in lines]

del lines
tokens

[[':begin', 'Seq_1'],
 ['hello()#1'],
 ['how_are_you()#1'],
 ['bye()#1'],
 [':end', 'Seq_1']]

In [101]:
uniq_names.clear()
# using existing alg

tr = Trace(alg)

BEGIN_MARKS = {'begin', 'begins', 'beginof', 'begin-of', 'begin_of',  }
END_MARKS = {'end', 'ends', 'endof', 'end-of', 'end_of',  }
MARK_SIGNS = (':', '.')
MARK_CHARS = ''.join(MARK_SIGNS)
COMMENT_SIGNS = ('#', '//')

begin_labels = []
end_labels = []

for line_i, line_tokens in enumerate(tokens):
    if not line_tokens:
        continue
        
    token = line_tokens[0]
    if token.startswith(COMMENT_SIGNS):
        continue
        
    rest_of_tokens = line_tokens[1:]
    if token.startswith(MARK_SIGNS):
        # labels line
        token = token.strip(MARK_CHARS).lower()
        if token in BEGIN_MARKS:
            begin_labels += rest_of_tokens
        elif token in END_MARKS:
            end_labels += rest_of_tokens
        else:
            line_n = line_i + 1
            raise SyntaxError("Unrecognized label mark: '%s'" % token+(' at line %d of trace' % line_n)+':\n'+("%d:\t%s\n" % (line_n, ' '.join(line_tokens)))
                              +' Expected one of:\n %s\n  or\n %s'%(str(BEGIN_MARKS),str(END_MARKS)))
#         print(token)
        if end_labels:
            # apply end_labels to previous act
            if not tr.acts:
                line_n = line_i + 1
                raise SyntaxError("Unexpected END labels: %s.\n No preceeding act found to attach these labels." % str(end_labels)+(' At line %d of trace' % line_n)+':\n'+("%d:\t%s\n" % (line_n, ' '.join(line_tokens))))
            
            prev_act = tr.acts[-1]   # no following acts written yet
            for lbl in end_labels:
                label = tr.parse_context_label(lbl).get_end()
                prev_act.end_labels.append(label)
            end_labels.clear()
    else:
        # act line
        if rest_of_tokens:
            line_n = line_i + 1
            raise SyntaxError("Not supported: multiple acts on the same line."+(' At line %d of trace' % line_n)+':\n'+("%d:\t%s\n" % (line_n, ' '.join(line_tokens))))
        
        # parse act token
        new_act = tr.parse_act_token(token)
        
        if begin_labels:
            # apply begin_labels to newly created act
            for lbl in begin_labels:
                label = tr.parse_context_label(lbl).get_begin()
                new_act.begin_labels.append(label)
        begin_labels.clear()
            
            
print('\n\o/')
print(' H')
print('/|')


===                Trace created.
===            hello()#1 created.
===              Seq_1#0 created.
===   Seq_1#0_BeginLabel created.
===      how_are_you()#1 created.
===              bye()#1 created.
===     Seq_1#0_EndLabel created.

\o/
 H
/|


In [102]:
_3ples = tr.get_triples()
print(len(_3ples))
_3ples

25


[('Trace', 'rdf:type', 'Trace'),
 ('Trace', 'hasOrigin', 'Algorithm'),
 ('Trace', 'hasFirstAct', 'hello()#1'),
 ('Trace', 'hasLastAct', 'bye()#1'),
 ('Trace', 'hasAct', 'hello()#1'),
 ('hello()#1', 'rdf:type', 'Act'),
 ('hello()#1', 'hasN', 1),
 ('hello()#1', 'hasBeginLabel', 'Seq_1#0_BeginLabel'),
 ('Seq_1#0_BeginLabel', 'rdf:type', 'BeginLabel'),
 ('Seq_1#0_BeginLabel', 'isLabelOf', 'Seq_1#0'),
 ('Seq_1#0', 'rdf:type', 'ActContext'),
 ('Seq_1#0', 'hasN', 0),
 ('Trace', 'hasAct', 'how_are_you()#1'),
 ('how_are_you()#1', 'rdf:type', 'Act'),
 ('how_are_you()#1', 'hasN', 1),
 ('hello()#1', 'hasNextAct', 'how_are_you()#1'),
 ('Trace', 'hasAct', 'bye()#1'),
 ('bye()#1', 'rdf:type', 'Act'),
 ('bye()#1', 'hasN', 1),
 ('bye()#1', 'hasEndLabel', 'Seq_1#0_EndLabel'),
 ('Seq_1#0_EndLabel', 'rdf:type', 'EndLabel'),
 ('Seq_1#0_EndLabel', 'isLabelOf', 'Seq_1#0'),
 ('Seq_1#0', 'rdf:type', 'ActContext'),
 ('Seq_1#0', 'hasN', 0),
 ('how_are_you()#1', 'hasNextAct', 'bye()#1')]

In [23]:
# обработать префиксы сущностей OWL
def make_namespace_prefix(name, default_prefix=':', known_prefixes=None):
    """ Prepends default_prefix to name if required and no one of known_prefixes present """
    if name.startswith(default_prefix):
        return name
    if name.startswith( tuple(known_prefixes or () ) ):
        return name
    # no known_prefix present
    return default_prefix + name
    

(make_namespace_prefix('rdf:type', known_prefixes=c2onto.OWLPredicate.values()) ,
make_namespace_prefix(':type', known_prefixes=c2onto.OWLPredicate.values()) ,
make_namespace_prefix('myType') ,
)


('rdf:type', ':type', ':myType')

In [24]:
f = lambda s: make_namespace_prefix(s, default_prefix=':', known_prefixes=c2onto.OWLPredicate.values())
triples = [tuple(f(a) for a in trpl) for trpl in triples]
triples

[(':Algorithm', 'rdf:type', ':Algorithm'),
 (':Algorithm', ':hasFunc', ':main()'),
 (':main()', 'rdf:type', ':Function'),
 (':main()', ':hasName', ':main'),
 (':main()', ':hasBody', ':Seq_1'),
 (':Seq_1', ':hasFirstSt', ':call-hello()'),
 (':Seq_1', ':hasLastSt', ':call-bye()'),
 (':Seq_1', ':hasSubStmt', ':call-hello()'),
 (':call-hello()', 'rdf:type', ':FuncCall'),
 (':Seq_1', ':hasSubStmt', ':call-how_are_you()'),
 (':call-how_are_you()', 'rdf:type', ':FuncCall'),
 (':call-hello()', ':hasNextSt', ':call-how_are_you()'),
 (':Seq_1', ':hasSubStmt', ':call-bye()'),
 (':call-bye()', 'rdf:type', ':FuncCall'),
 (':call-how_are_you()', ':hasNextSt', ':call-bye()')]

In [26]:
# И формировать SPARQL-запросы.
def triple_to_sparql_insert(triple, prefix_str=""):

    s = """INSERT DATA
{ 
%s %s %s .
}""" % triple
    return prefix_str + s
    

In [27]:
prefix_str = """
BASE <http://vstu.ru/poas/se/c_schema_2020-01#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

"""
q = triple_to_sparql_insert(triples[0], prefix_str)
print(q)


BASE <http://vstu.ru/poas/se/c_schema_2020-01#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

INSERT DATA
{ 
:Algorithm rdf:type :Algorithm .
}


In [ ]:
# from SPARQLWrapper import SPARQLWrapper, JSON, RDFXML, N3


In [47]:
%%timeit

tuple( [] )

305 ns ± 5.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [48]:
%%timeit

tuple( () )

271 ns ± 7.08 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [62]:
v = FuncDefFinder()

In [ ]:
v.generic_visit(ast)

In [18]:
isinstance('abc',str)

True

In [53]:
alg.node_name


'Algorithm@<root>'

In [38]:
alg.nodes

[<__main__.FuncDefNode at 0x5eb1a50>, <__main__.FuncDefNode at 0x5eb19f0>]

In [39]:
# def rpr(self):
#     return self.repr()
# alg.nodes[0].__str__ = rpr

alg.nodes

[<__main__.FuncDefNode at 0x5eb1a50>, <__main__.FuncDefNode at 0x5eb19f0>]

In [40]:
f = alg.nodes[0]


In [42]:
f.node_name

'FuncDef@examples/ex2f.c:3:6'

In [30]:
b = ast.ext[1].body
b2 = b.block_items[1]

In [68]:
coord2str(b2.coord)

'7:1'

In [71]:
type(b2.coord)

pycparser.plyparser.Coord

In [44]:
if1 = b2.block_items[0]

In [54]:
type(if1.cond)

pycparser.c_ast.Constant

In [63]:
print(ast_to_code(if1.cond))

1


In [65]:
call = b.block_items[0].stmt
print(ast_to_code(call.args))

a, b, c, d, e, f


In [37]:
type(type(b))

type

In [40]:
type(b) == c_ast.Compound

True

In [35]:
b.__class__ == c_ast.Compound

True

In [53]:
ast.show(nodenames=1, attrnames=True)


FileAST: 
  Decl <ext[0]>: name=global_counter, quals=[], storage=[], funcspec=[]
    TypeDecl <type>: declname=global_counter, quals=[]
      IdentifierType <type>: names=['int']
    Constant <init>: type=int, value=0
  FuncDef <ext[1]>: 
    Decl <decl>: name=main, quals=[], storage=[], funcspec=[]
      FuncDecl <type>: 
        TypeDecl <type>: declname=main, quals=[]
          IdentifierType <type>: names=['void']
    Compound <body>: 
      For <block_items[0]>: 
        FuncCall <stmt>: 
          ID <name>: name=st_3
          ExprList <args>: 
            ID <exprs[0]>: name=a
            ID <exprs[1]>: name=b
            ID <exprs[2]>: name=c
            ID <exprs[3]>: name=d
            ID <exprs[4]>: name=e
            ID <exprs[5]>: name=f
      Compound <block_items[1]>: 
        If <block_items[0]>: 
          Constant <cond>: type=int, value=1
          Compound <iftrue>: 
            If <block_items[0]>: 
              Constant <cond>: type=int, value=0
              C